In [1]:
import numpy as np
from sklearn import svm
import torchvision
import torch
import torch.nn as nn 
import torch.optim as optim
import torch.functional as F
import torchvision.models as pretrained_models
import random
import struct

In [2]:
def ids_by_label(ids, labels_list, target, labels_set, shuffle):
    if not isinstance(shuffle,bool):
        raise TypeError("shuffle should be a bool")
    pairs = list(zip(ids,labels_list))
    if labels_set != 'all':
        pairs = [(_id, label) for _id,label in pairs if label.item() in labels_set]
    if shuffle:
        random.shuffle(pairs)
        
    return [_id for _id,_label in pairs if _label.item()==target], [_id for _id,_label in pairs if _label.item()!=target]

In [3]:
def check_if_target(indices, labels, target):
    for idx in indices:
        if labels[idx].item() == target:
            return True 
    return False

In [4]:
def make_bags(target_ids, non_target_ids, labels, bag_size, num_bags, prob, target):
    full_list = target_ids + non_target_ids 
    for _ in range(num_bags):
        random.shuffle(full_list)
        random.shuffle(non_target_ids)
        toss = np.random.rand()
        if toss <= prob:
            random.shuffle(target_ids)
            rest = full_list[:bag_size-1]
            rest.append(target_ids[0])
            assert check_if_target(rest, labels, target) 
            yield rest, [1]
        else:
            rest = non_target_ids[:bag_size]
            assert not check_if_target(rest,labels,target) 
            yield rest, [0]

In [5]:
def get_instance(indices,images):
    instance_set = []
    for idx in indices:
        instance_set.append(images[idx])

    instance_set = torch.stack(instance_set,dim=0)
    
    return instance_set

In [6]:
if __name__ == '__main__':
    mnist = torchvision.datasets.MNIST(root='',download=True)
    images = mnist.data #shape = (60000,28,28)
    labels = mnist.targets #shape = (60000)
    ids = np.arange(len(labels))

    BAG_SIZE = 100
    PROB = 1
    NUM_BAGS = 600
    TARGET = 0 or 7

    target_ids, non_target_ids = ids_by_label(ids, labels, 0 or 7,'all', True)

    new_bag = make_bags(target_ids, non_target_ids, labels, BAG_SIZE, NUM_BAGS, PROB, TARGET)
    sample_bag, sample_label = next(new_bag)
    instance_set = get_instance(sample_bag,images)
    print(instance_set)

100
1
100
